# Span Detection UA only implementation

1. Import dependencies

In [1]:
import json

import pandas as pd

from src.definitions import (
    RAW_DATA_FOLDER,
    PROMPTS_FOLDER,
)

2. Prepare Env

In [2]:
generated_examples_count = 5
max_content_length = 4000

In [3]:
techniques =[
    "appeal_to_fear",
    "bandwagon",
    "cherry_picking",
    "cliche",
    "euphoria",
    "fud",
    "glittering_generalities",
    "loaded_language",
    "straw_man",
    "whataboutism",
]

3. Load dataset

In [4]:
df = pd.read_parquet(RAW_DATA_FOLDER / "span-detection.parquet")

3. Extract ua and ru parts of dataset 

In [5]:
df_ua = df.query("lang == 'uk' and manipulative == True")
df_ru = df.query("lang == 'ru' and manipulative == True")

4. Create prompts folder

In [6]:
UA_PROMPTS_FOLDER = PROMPTS_FOLDER / "ua"
RU_PROMPTS_FOLDER = PROMPTS_FOLDER / "ru"

UA_PROMPTS_FOLDER.mkdir(parents=True, exist_ok=True)
RU_PROMPTS_FOLDER.mkdir(parents=True, exist_ok=True)

In [7]:
def save_prompt(file_name, content):
    with open(file_name, "w") as text_file:
        text_file.write(content)

5. Define base prompt

In [8]:
def example_template(df, i):
    loc = df.iloc[i].to_dict()
    content = loc["content"].replace("\n", " ")
    spans = [tuple(it.tolist()) for it in loc["trigger_words"]]
    techniques = loc["techniques"]

    return \
f"""\
content {i}: {content}
spans {i}: {spans}
techniques {i}: {techniques}
"""

def convert_to_example(df):
    return [example_template(df, i) for i in range(len(df))]

5. Define ua prompt

In [9]:
base_prompt_ua = f"""\
Згенеруйте розмічені дані для класифікації токенів в задачі пошуку маніпуляцій у постах соціальних мереж.  
Кожен приклад має містити:  
1. content - Текст посту чи коментаря (максимум {max_content_length} символів) з соціальної мережі Телеграм. 
2. spans - Список індексних діапазонів символів маніпуляції.  
3. techniques - Список маніпуляцій використаних у тексті посту. Можливі значення: 'appeal_to_fear', 'bandwagon', 'cherry_picking', 'cliche', 'euphoria', 'fud', 'glittering_generalities', 'loaded_language', 'straw_man', 'whataboutism'
"""

def ua_prompt_template(examples):
    examples_list_str = "##\n".join(examples)
    examples_str = f"Приклади:\n{examples_list_str}"
    command_str = f"Тепер згенеруйте ще {generated_examples_count} таких прикладів:"

    return base_prompt_ua + examples_str + command_str

ua_prompt = ua_prompt_template(convert_to_example(df_ua))

save_prompt(UA_PROMPTS_FOLDER / f"generate-data-all.txt", ua_prompt) 

for tech in techniques:
    df_tech_ua = df_ua[df_ua["techniques"].str.contains(tech, regex=False)]
    ua_prompt = ua_prompt_template(convert_to_example(df_tech_ua))

    save_prompt(UA_PROMPTS_FOLDER / f"generate-data-{tech}.txt", ua_prompt)

6. Define ru prompt

In [10]:
base_prompt_ru = f"""\
Сгенерируйте размеченные данные для классификации токенов в задаче поиска манипуляций в постах социальных сетей.
Каждый пример должен содержать:
1. content – текст поста или комментария (максимум {max_content_length} символов) из социальной сети Телеграмм.
2. spans – список индексных диапазонов символов манипуляций.
3. techniques – Список манипуляций использованных в тексте поста. Возможные значения: 'appeal_to_fear', 'bandwagon', 'cherry_picking', 'cliche', 'euphoria', 'fud', 'glittering_generalities', 'loaded_language', 'straw_man', 'whataboutism'
"""

def ru_prompt_template(examples):
    examples_list_str = "##\n".join(examples)
    examples_str = f"Примеры:\n{examples_list_str}"
    command_str = f"Теперь сгенерируйте еще {generated_examples_count} таких примеров:"

    return base_prompt_ru + examples_str + command_str

ru_prompt = ru_prompt_template(convert_to_example(df_ru))
save_prompt(RU_PROMPTS_FOLDER / f"generate-data-all.txt", ru_prompt) 

for tech in techniques:
    df_tech_ru = df_ru[df_ru["techniques"].str.contains(tech, regex=False)]
    ru_prompt = ru_prompt_template(convert_to_example(df_tech_ru))

    save_prompt(RU_PROMPTS_FOLDER / f"generate-data-{tech}.txt", ru_prompt) 

7. Create Vertex AI examples

In [11]:
def convert_row_to_json_example(df, i):
    loc = df.iloc[i].to_dict()
    content = loc["content"].replace("\n", " ")
    spans = [tuple(it.tolist()) for it in loc["trigger_words"]]
    techniques = loc["techniques"]

    return {"inputs": [str(techniques), str(spans)], "outputs": [content]}

def convert_df_to_json_example(df):
    return [convert_row_to_json_example(df, i) for i in range(len(df))]

def save_examples_json(path, df):
    with open(path, "w", encoding="utf8") as f:
        json.dump(convert_df_to_json_example(df), f, ensure_ascii=False)

In [12]:
save_examples_json(UA_PROMPTS_FOLDER / "generate-data-examples.json", df_ua)

In [13]:
save_examples_json(RU_PROMPTS_FOLDER / "generate-data-examples.json", df_ru)

8. Create DS for fine tunning

In [14]:
def convert_row_to_json_ds(df, i, instruction):
    loc = df.iloc[i].to_dict()
    content = loc["content"].replace("\n", " ")
    spans = [tuple(it.tolist()) for it in loc["trigger_words"]]
    techniques = loc["techniques"]
    input_text = f"{techniques} ## {spans}"

    return {
        "systemInstruction": {"role": "user", "parts": [{"text": instruction}]},
        "contents": [
            {
                "role": "user",
                "parts": [
                    {
                        "text": input_text,
                    }
                ],
            },
            {
                "role": "model",
                "parts": [
                    {
                        "text": content,
                    }
                ],
            },
        ],
    }


def convert_df_to_json_ds(df, instruction):
    return [convert_row_to_json_ds(df, i, instruction) for i in range(len(df))]


def save_ds_json(path, df, instruction):
    with open(path, "w", encoding="utf8") as f:
        for it in convert_df_to_json_ds(df, instruction):
            json.dump(it, f, ensure_ascii=False)
            f.write("\n")

In [15]:
instruction_ua = f"""\
Згенеруйте розмічені дані для класифікації токенів в задачі пошуку маніпуляцій у постах соціальних мереж.
Формат питання - $techniques ## $spans де:
techniques - Список маніпуляцій використаних у тексті посту. Можливі значення: 'appeal_to_fear', 'bandwagon', 'cherry_picking', 'cliche', 'euphoria', 'fud', 'glittering_generalities', 'loaded_language', 'straw_man', 'whataboutism'
spans - Список індексних діапазонів символів маніпуляції.
Відповідь - Текст посту чи коментаря (максимум {max_content_length} символів) з соціальної мережі Телеграм."""

save_ds_json(UA_PROMPTS_FOLDER / "generate-data-ds.jsonl", df_ua, instruction_ua)

In [16]:
instruction_ru = f"""\
Сгенерируйте размеченные данные для классификации токенов в задаче поиска манипуляций в постах социальных сетей.
Формат вопроса - $techniques ## $spans где:
techniques – Список манипуляций использованных в тексте поста. Возможные значения: 'appeal_to_fear', 'bandwagon', 'cherry_picking', 'cliche', 'euphoria', 'fud', 'glittering_generalities', 'loaded_language', 'straw_man', 'whataboutism'
spans – список индексных диапазонов символов манипуляции.
Ответ – текст поста или комментария (максимум {max_content_length} символов) из социальной сети телеграмм."""

save_ds_json(RU_PROMPTS_FOLDER / "generate-data-ds.jsonl", df_ru, instruction_ru)